In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('..')

In [2]:
from nflows.distributions import (
    StandardNormal
)
from src.models.dists import (
    create_affine_coupling,
    create_lu_linear,
)
from nflows import transforms
from nflows.flows import Flow
import torch

In [3]:
n_dim = 4
context_features = 16
n_flow_steps = 2

base_dist = StandardNormal((n_dim,))
transform = transforms.CompositeTransform(
    [
        transforms.CompositeTransform([
            create_lu_linear(n_dim),
            create_affine_coupling(
                n_dim,
                i,
                context_features=context_features,
                n_hidden=20
            )
        ]) for i in range(n_flow_steps)
    ]
)

transform = transforms.CompositeTransform([transform, create_lu_linear(n_dim)])

flow = Flow(transform, base_dist)

In [4]:
output = flow.sample(1, context=torch.randn(5, 16))

In [5]:
output.shape

torch.Size([5, 1, 4])

In [6]:
log_prob = flow.log_prob(torch.randn(5, 4), context=torch.randn(5, 16))

In [7]:
log_prob.shape

torch.Size([5])

In [11]:
output = flow.sample(5)

In [12]:
output.shape

torch.Size([5, 4])